In [149]:
import pyspark.sql.functions as sqlf 
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))
import time
# Create a SparkSession
spark = SparkSession.builder \
    .appName("Display Mental Health Dataset") \
    .getOrCreate()

In [160]:

# Start time
start_time = time.time()
# Load the CSV file into a DataFrame
df = spark.read.csv("C:\\Users\\User\\Downloads\\Mental_Health_Dataset.csv", header=True)
print("The dataset has", df.count(), 'data objects')
# End time
end_time = time.time()

# Calculate processing time
processing_time = end_time - start_time
print(f"Processing time: {round(processing_time, 3)} seconds")

The dataset has 292364 data objects
Processing time: 0.192 seconds


In [161]:
df.show()

+---------------+------+--------------+----------+-------------+--------------+---------+------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|      Timestamp|Gender|       Country|Occupation|self_employed|family_history|treatment|Days_Indoors|Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|
+---------------+------+--------------+----------+-------------+--------------+---------+------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|8/27/2014 11:29|Female| United States| Corporate|         NULL|            No|      Yes|   1-14 days|           Yes|            No|                  Yes|     Medium|              No|           No|            Yes|                     No|    Not su

In [162]:
start_time = time.time()

# Filter the DataFrame to include only rows where the country is 'United Kingdom'
uk_data = df.filter(df['Country'] == 'United Kingdom')
end_time = time.time()

# Calculate processing time
processing_time = end_time - start_time

print(f"Processing time: {round(processing_time, 4)} seconds")

# Display the filtered DataFrame
uk_data.show()

Processing time: 0.002 seconds
+---------------+------+--------------+----------+-------------+--------------+---------+----------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|      Timestamp|Gender|       Country|Occupation|self_employed|family_history|treatment|    Days_Indoors|Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|
+---------------+------+--------------+----------+-------------+--------------+---------+----------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|8/27/2014 12:51|Female|United Kingdom| Corporate|           No|            No|       No|       1-14 days|           Yes|            No|                  Yes|     Medium|              No|           No|   

In [165]:
print("Number of data objects in the dataset:", uk_data.count())

Number of data objects in the dataset: 51404


In [172]:
# Count null values in each column
start_time = time.time()

null_counts = uk_data.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in uk_data.columns])
if null_counts:
    print("There are NULL values in the dataset.")
else:
    print("There are no NULLS values in the dataset.")
end_time = time.time()

processing_time = end_time - start_time

print(f"Processing time: {round(processing_time, 4)} seconds")

# Display sum of nulls in each column
null_counts.show()

There are NULL values in the dataset.
Processing time: 0.0264 seconds
+---------+------+-------+----------+-------------+--------------+---------+------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|Timestamp|Gender|Country|Occupation|self_employed|family_history|treatment|Days_Indoors|Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|
+---------+------+-------+----------+-------------+--------------+---------+------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|        0|     0|      0|         0|          780|             0|        0|           0|             0|             0|                    0|          0|               0|            0|              0|                

In [145]:
# Calculate null percentages for each column
total_rows = uk_data.count()
null_percentages = [(col_name, (uk_data.filter(col(col_name).isNull()).count() / total_rows) * 100)
                    for col_name in uk_data.columns
                    if uk_data.filter(col(col_name).isNull()).count() > 0]

# Print null percentages for columns with null values
for col_name, null_percentage in null_percentages:
    print(f"Column '{col_name}' has : {round(null_percentage, 3)} % null")

Column 'self_employed' has : 1.517 % null


In [137]:
# Task 1: Check and print the number of data objects in the CSV file
num_data_objects = uk_data.count()
print("Number of data objects in the CSV file before dropping columns:", num_data_objects)

UK_Data_NoNull = uk_data.dropna()

# Task 2: Check and print the number of data objects in the CSV file after dropping columns
num_data_objects_after_drop = UK_Data_NoNull.count()
print("Number of data objects in the CSV file after dropping columns:", num_data_objects_after_drop)

Number of data objects in the CSV file before dropping columns: 51404
Number of data objects in the CSV file after dropping columns: 50624


In [98]:
# Count null values in each column
null_counts = UK_Data_NoNull.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in UK_Data_NoNull.columns])
# Display sum of nulls in each column
null_counts.show()

+---------+------+-------+----------+-------------+--------------+---------+------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|Timestamp|Gender|Country|Occupation|self_employed|family_history|treatment|Days_Indoors|Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|
+---------+------+-------+----------+-------------+--------------+---------+------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|        0|     0|      0|         0|            0|             0|        0|           0|             0|             0|                    0|          0|               0|            0|              0|                      0|           0|
+---------+------+-------+----------+-----------

In [138]:
# Identify duplicate rows
duplicate_rows = UK_Data_NoNull.groupBy(UK_Data_NoNull.columns).count().where('count > 1')

# Count total number of duplicate rows
total_duplicates = duplicate_rows.count()

# Show all duplicate rows
if total_duplicates > 0:
    print("Total number of duplicate rows:", total_duplicates)
    duplicate_rows.show(truncate=False)
else:
    print("No duplicate rows found.")

Total number of duplicate rows: 48
+---------------+------+--------------+----------+-------------+--------------+---------+------------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+-----+
|Timestamp      |Gender|Country       |Occupation|self_employed|family_history|treatment|Days_Indoors      |Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|count|
+---------------+------+--------------+----------+-------------+--------------+---------+------------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+-----+
|8/27/2014 13:47|Female|United Kingdom|Corporate |No           |No            |Yes      |31-60 days        |No            |No            |Maybe                |High       |No  

In [126]:
# Calculate total number of rows
total_rows = UK_Data_NoNull.count()
# Calculate percentage of duplicates
percentage_duplicates = (total_duplicates / total_rows) * 100
print(f"Percentage of duplicate rows: {round(percentage_duplicates, 3)}% ")

Percentage of duplicate rows: 0.095% 


In [139]:
# Drop duplicate rows
print("Before removing duplicates:",UK_Data_NoNull.count() )
uk_data_no_duplicates = UK_Data_NoNull.dropDuplicates()

# Check for duplicates in the new DataFrame
total_duplicates_after_drop = uk_data_no_duplicates.groupBy(uk_data_no_duplicates.columns).count().where('count > 1').count()

if total_duplicates_after_drop == 0:
    print("No duplicate rows remaining after dropping duplicates.")
else:
    print("Duplicate rows still exist after dropping duplicates.")

print('Total number of rows after removing duplicates is ', uk_data_no_duplicates.count())

Before removing duplicates: 50624
No duplicate rows remaining after dropping duplicates.
Total number of rows after removing duplicates is  50576


In [140]:
Removed_Date_Data = uk_data_no_duplicates.drop('Timestamp')
Removed_Date_Data.show()

+------+--------------+----------+-------------+--------------+---------+------------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|Gender|       Country|Occupation|self_employed|family_history|treatment|      Days_Indoors|Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|
+------+--------------+----------+-------------+--------------+---------+------------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|Female|United Kingdom|  Business|           No|            No|      Yes|More than 2 months|           Yes|            No|                   No|       High|              No|           No|          Maybe|                     No|          No|
|Female|United Kingdom|   Student|  

In [141]:
Cleaned_UK_Mental_Health = Removed_Date_Data
Cleaned_UK_Mental_Health.show()

+------+--------------+----------+-------------+--------------+---------+------------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|Gender|       Country|Occupation|self_employed|family_history|treatment|      Days_Indoors|Growing_Stress|Changes_Habits|Mental_Health_History|Mood_Swings|Coping_Struggles|Work_Interest|Social_Weakness|mental_health_interview|care_options|
+------+--------------+----------+-------------+--------------+---------+------------------+--------------+--------------+---------------------+-----------+----------------+-------------+---------------+-----------------------+------------+
|Female|United Kingdom|  Business|           No|            No|      Yes|More than 2 months|           Yes|            No|                   No|       High|              No|           No|          Maybe|                     No|          No|
|Female|United Kingdom|   Student|  